In [3]:
import numpy as np
import matplotlib.pyplot as plt
from ORRFA import *
import interpretableai
from interpretableai import iai
import pandas as pd
import json
from HR import *

In [15]:
dataset.index

RangeIndex(start=0, stop=308, step=1)

In [26]:
dataset = pd.read_csv("climate_change.csv")

train_idx = np.random.choice(a = np.arange(0, len(dataset)), size = int(0.6*len(dataset)))
test_idx = [i for i in np.arange(0, len(dataset)) if i not in train_idx]

features = dataset.loc[train_idx].iloc[:, :-2]
diagnosis = dataset.loc[train_idx].iloc[:, -1]

orig_features_size = features.copy().shape[1]

In [28]:
print(len(train_idx))
print(len(test_idx))

184
178


In [23]:
features_test = dataset.loc[test_idx].iloc[:, :-2]
diagnosis_test = dataset.loc[test_idx].iloc[:, -1]

In [24]:
print(len(dataset))
print(len(features))
print(len(features_test))

308
184
170


In [5]:
features

,Year,Month,MEI,CO2,CH4,N2O,CFC_11,CFC_12,TSI
54,1987,11,1.253,347.96,1716.57,306.393,239.588,433.915,1365.9837
154,1996,3,-0.264,364.17,1769.70,311.828,269.482,528.589,1365.5815
32,1986,1,-0.307,346.54,1675.82,305.775,218.382,396.082,1365.6746
158,1996,7,-0.208,363.53,1747.03,311.481,268.803,528.804,1365.5968
250,2004,3,-0.125,378.73,1807.48,318.198,254.712,543.008,1366.1272
...,...,...,...,...,...,...,...,...,...
274,2006,3,-0.591,382.66,1786.77,319.902,250.155,540.952,1365.7850
130,1994,3,0.159,359.91,1754.63,310.030,270.676,515.302,1366.0304
243,2003,8,0.241,374.32,1773.56,317.703,255.889,542.529,1366.2744
98,1991,7,1.011,356.12,1730.94,308.789,267.250,493.348,1366.6668


In [ ]:
def gen_features(sub_paths, data, features, orig_feature_size):
    new_features = features.copy()
    for path in sub_paths:
        statement = gen_statement(path, data, features)
        if statement != '(0 < 0.0)' and statement != '(0 >= 0.0)':

            try:
                index_feature = features.loc[eval(statement)].index


                new_feature = [1 if i in index_feature else 0 for i in range(len(features))]

                if sum(new_feature) >= 1:
                    new_features[statement] = new_feature

            except:
                print(statement)
    return new_features.iloc[:, orig_feature_size:]

In [ ]:
np.random.seed(1)
n = 5
rules = {}
for i in range(1,15):
    #################
    dataset = dataset.sample(n=int(n),axis='columns')
    OCT_H = iai.GridSearch(
          iai.OptimalTreeRegressor(
              max_depth = 3, cp = 0.00000000001))

    OCT_H.fit(features, diagnosis)
    OCT_H.write_json(f"tre_{i}.json")

    ######################
    f = open(f"tre_{i}.json")
    data = json.load(f)

    paths = gen_paths(data)

    rule_features_new = gen_features(
            paths, data, features, orig_features_size)

    rules[f"tre_{i}"] = rule_features_new

    print(f"Tree: {i}")


In [ ]:
features.columns

In [ ]:
big_df = pd.DataFrame()
for tree in rules.keys():

    big_df = pd.concat([big_df, rules[tree]],axis =1)


In [ ]:
df = big_df.loc[:,~big_df.columns.duplicated()].copy()

In [ ]:
df

In [ ]:
for column in df.columns[1:]:
    if df[column].sum() == len(df):
        del df[column]


In [ ]:
df

In [ ]:
features_for_robust_reg = df

In [ ]:
rule_features = features_for_robust_reg.copy()
features_for_robust_reg = pd.concat([features, features_for_robust_reg], axis = 1)

In [ ]:
model = HolisticRobust(
                       α = 0,
                       ϵ = 0,
                       r = 0.05,
                       classifier = "Linear",
                       learning_approach = "HR") # Could be ERM either

In [ ]:
features.insert(loc = 0, column = 'Intecept', value = [1 for i in range(len(features_for_robust_reg))])

In [ ]:
θ, obj_value = model.fit(ξ = (np.matrix(features_for_robust_reg), np.array(diagnosis)))

In [ ]:
θ

In [ ]:
predictions = np.multiply(θ, features_for_robust_reg
                          ).sum(axis = 1)

In [ ]:
plt.plot(diagnosis - predictions)

In [ ]:
(diagnosis - predictions).describe()

In [ ]:
SSE_model = sum((diagnosis - predictions)**2)
SSE_mean = sum((diagnosis - diagnosis.mean())**2)

1 - SSE_model/SSE_mean

In [ ]:
features['CFC_12'].describe()

In [ ]:
features

In [ ]:
rule_features

In [ ]:
features = features_test

for rule in rule_features.columns:

    index_feature = features.loc[eval(rule)].index

    new_feature = [1 if i in index_feature else 0 for i in range(len(features))]

    if sum(new_feature) >= 1:
        features[rule] = new_feature


In [ ]:
features

In [ ]:
θ